# **Projet trouve ton job**

## **Imports**
---

In [ ]:
import numpy as np
import pandas as pd
import os
import unicodedata

# mount GoogleDrive and set the files path
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/Projet Python - Trouve ton job'

## **Déclaration des fonctions**
---

In [ ]:
tab = pd.read_json("data.json")

def extract_jsonTab(tab, nb_vlue):
    df1=pd.DataFrame()
    df2=pd.DataFrame()
    for elem in tab:
        x=0 
        vlue=[""] * nb_vlue
        for val in  elem:        
            val=val.replace("\n","")
            vlue[x]=val
            x+=1
        df1=pd.DataFrame([pd.Series(vlue)])
        # df1=pd.concat([df1,pd.DataFrame([[vlue[0],vlue[1]]])], ignore_index=True)
        df2= pd.concat([df2,df1], ignore_index=True)
            
    return df2

In [ ]:
def serialisation_colonnes_competences(df):
    df1=pd.DataFrame()
    df2=pd.DataFrame()
    for i in df.index: 
        chn=''
        for j in range(len(df.columns)):   
            chn=chn +df[j][i]
            if j != (len(df.columns)-1):
                chn=chn +", "
        df1=pd.DataFrame([pd.Series(chn)])   
        df2= pd.concat([df2,df1], ignore_index=True)
    return df2

In [ ]:
def réecriture_des_dates_de_publication (df_in):
    df_concat = pd.DataFrame()
    df_date = pd.DataFrame()
    
    for i in df_in:
        i=i.replace("\n","")
        if i == 'postée hier':
            i = pd.Timestamp(2023,1,15)-pd.Timedelta(days=1)
            # print(i)
        else:
            x=float(i.split()[4])
            if i.find('jours') != -1:
                i = pd.Timestamp(2023,1,15)-pd.Timedelta(days=x)
                # print (i.strftime('%Y-%m-%d'))
            elif i.find('mois') != -1:
                i = pd.Timestamp(2023,1,15)-pd.Timedelta(days=x*30)
                # print(i.strftime('%Y-%m-%d'))
            else:
                i = pd.Timestamp(2023,1,15)-pd.Timedelta(hours=x)
                # print(i.strftime('%Y-%m-%d'))
        df_concat = pd.DataFrame([pd.Series(i.strftime('%Y-%m-%d'))])
        df_date = pd.concat([df_date,df_concat], ignore_index=True)
    return df_date

In [ ]:
def recuperation_salaire_minimum (df):
    df_concat = pd.DataFrame()
    df_salaire = pd.DataFrame()
    for i in df: 
        if i == '':
            i=""
            # print ("aucune valeure")
        else:
            i=i.split('an')[0].replace('€', '').replace('/', '').split('-')[0].replace(',00', '').replace('.', '')
            i=int(i)
            # print(type(i))
        df_concat = pd.DataFrame([pd.Series(i)])
        df_salaire = pd.concat([df_salaire,df_concat], ignore_index=True)
    return df_salaire

In [ ]:
def recuperation_salaire_maximum (df):
    df_concat = pd.DataFrame()
    df_salaire = pd.DataFrame()    
    for i in df: 
        if i == '':
            i=""
            # print ("aucune valeur")
        else:
            i=i.split('an')[0].replace('€', '').replace('/', '').split('-')[1].replace(',00', '').replace('.', '')
            i=int(i)
            # print(type(i))
        df_concat = pd.DataFrame([pd.Series(i)])
        df_salaire = pd.concat([df_salaire,df_concat], ignore_index=True)
    return df_salaire

In [ ]:
from pandas.io.formats.format import List
def labelisations_colonne_intitule_poste(tab_origin):

    x=0
    
    df1=pd.DataFrame()
    df2=pd.DataFrame()
    val=[]   
    for lng in tab_origin.index:

        #extraction de la colonne Intitulé du poste
        val = tab_origin.iloc[lng]['Intitulé du poste'][0]
    
        #suppression de H/F...    
        # val = val.replace('(', '').replace(')', '').replace('H/F', '').replace('F/H', '').replace('h/f', '')
      
        #simplifier le traitement en passant tout en minuscule

        job=""  
        class_job=""
        niveau=""
        #Ce sont les intitulés en rapport avec les annoces pour la data 
        if val.find('data')!=-1 or val.find('données')!=-1 or val.find('virtualisation')!=-1 or val.find('digital')!=-1:
            
            class_job="Data"
            
            if val.find('analyst')!=-1 or val.find('analytics')!=-1 or val.find('analyses')!=-1  or val.find('analyse')!=-1 :
                job = "Data analyst" 
                # print(str(lng) + "  "+val)
            
            elif val.find('scientist')!=-1 or val.find('datascience')!=-1:
                job = "Data scientist"
                # print(str(lng) + "  "+val)
            
            elif val.find('manager')!=-1 or val.find('management')!=-1:
                job = "Data manager"
                # print(str(lng) + "  "+val)
            
            elif val.find('contrôleur')!=-1:
                job = "Data controler"
                # print(str(lng) + "  "+val)
            
            elif val.find('big data')!=-1 or val.find('bigdata')!=-1: 
                job = "Big Data"
                #print(str(lng) + "  "+val)
            
            elif val.find('ingenieur')!=-1 or val.find('engineer')!=-1 or val.find('ingénieur')!=-1:
                job = "Data ingenier"
                # print(str(lng) + "  "+val)
            
            elif val.find('architecte')!=-1:
                job = "Data architecte"
                # print(str(lng) + "  "+val)
            
            elif val.find('consultant')!=-1:
                job = "Data consultant"
                # print(str(lng) + "  "+val)
            
            elif val.find('viz')!=-1 or val.find('visualisation')!=-1 or val.find('tableau de bord')!=-1:
                job = "Data viz"
                # print(str(lng) + "  "+val)
            
            elif val.find('personnelles')!=-1 or val.find('protection et digital')!=-1:
                job = "DPO"
                # print(str(lng) + "  "+val)
            
            elif val.find('chef de projet')!=-1:
                job = "Chef de projet data"
                # print(str(lng) + "  "+val)
            
            else:
                job = "Data other"  
                # print(str(lng) + "  "+val)
            
        #Ce sont les intitulés en rapport avec les annoces pour le BI     
        elif val.find('business')!=-1 or val.find('bi')!=-1 or val.find('msbi')!=-1 or val.find('sap')!=-1 or val.find('marketing')!=-1 or val.find('décisionnel')!=-1 or val.find('talend')!=-1 or  val.find('cognos')!=-1 or  val.find('anaplan')!=-1 or  val.find('customer')!=-1:
            
            class_job="Business"
            # print(str(lng) + "  "+val)
            
            if val.find('analyst')!=-1 :
                job = "Business analyst" 
                # print(str(lng) + "  "+val)
                
            elif val.find('intelligence')!=-1 or val.find('bi')!=-1 or val.find('msbi')!=-1 or val.find('cognos')!=-1 or val.find('anaplan')!=-1 or val.find('sap')!=-1 or val.find('talend')!=-1:
                job = "Business intelligence" 
                # print(str(lng) + "  "+val)
                
            elif val.find('marketing')!=-1:
                job = "Marketing" 
                # print(str(lng) + "  "+val)
                
            else:
                job = "Business other"  
                #print(str(lng) + "  "+val)
                

        #Ce sont les intitulés en rapport avec les annoces pour le cloud         
        elif val.find('cloud')!=-1:
            class_job="Net"
            
            if val.find('sécurité')!=-1:
                job = "Cyber cloud" 
                # print(str(lng) + "  "+val)
                
            elif val.find('cloud')!=-1:
                job = "Ingenieurie cloud" 
                # print(str(lng) + "  "+val)
                
            else:
                job = "Net other"  
                # print(str(lng) + "  "+val)
            
        #Ce sont les intitulés en rapport avec les annoces pour developpeur 
        elif val.find('developpeur')!=-1 or val.find('développeur')!=-1 or val.find('logiciel')!=-1 or val.find('windows')!=-1 or val.find('web')!=-1 or val.find('unix/linux')!=-1 or val.find('developer')!=-1 or val.find('devops')!=-1 or val.find('développement')!=-1 or val.find('test')!=-1 or val.find('application')!=-1 or val.find('informatique')!=-1 or val.find('applicatif')!=-1:
            
            class_job="Génie informatique"
            # print(str(lng) + "  "+val)
            
            if val.find('web')!=-1 or val.find('angular')!=-1 or  val.find('php')!=-1 or  val.find('j2ee')!=-1:
                 job = "Developpement web" 
                 # print(str(lng) + "  "+val)
                
            elif val.find('test')!=-1 :
                job = "Developpement test" 
                # print(str(lng) + "  "+val)
                
            elif val.find('application')!=-1 or val.find('ios')!=-1 or val.find('applicatif')!=-1:
                job = "Developpement application" 
                # print(str(lng) + "  "+val)
       
            elif val.find('logiciel')!=-1 or val.find('software')!=-1 or val.find('windows')!=-1 or val.find('linux')!=-1 or val.find('etude')!=-1 or val.find('vba')!=-1:
                job = "Developpement logiciel" 
                # print(str(lng) + "  "+val)
                
            elif val.find('agile')!=-1 or val.find('devops')!=-1:
                job = "Developpement agile" 
                # print(str(lng) + "  "+val)
            
            elif val.find('full')!=-1 :
                job = "Developpement full stack" 
                # print(str(lng) + "  "+val)
            
            elif val.find('backend')!=-1 :
                job = "Developpement backend" 
                # print(str(lng) + "  "+val)
                       
            elif val.find('développeur')!=-1 or val.find('developpeur')!=-1 :
                job = "Developpement logiciel" 
                # print(str(lng) + "  "+val)
                
            elif val.find('technicien')!=-1 :
                job = "Technicien informatique" 
                # print(str(lng) + "  "+val)
               
            else:
                job = "Business other"  
                # print(str(lng) + "  "+val)
            
            
        #Ce sont les intitulés en rapport avec les reseaux autres que le cloud     
        elif val.find('réseau')!=-1 or val.find('cloud')!=-1 or val.find('cyber')!=-1 or val.find('middleware')!=-1 or val.find('vmware')!=-1 or val.find('cybersoc')!=-1 or val.find('infrastructure')!=-1:
            
            class_job="Net"
            # print(str(lng) + "  "+val)
            
            if val.find('cyber')!=-1 or val.find('sécurité')!=-1:
                job = "Cyber" 
                # print(str(lng) + "  "+val)
                
            elif val.find('middleware')!=-1 :
                job = "Middleware"
                # print(str(lng) + "  "+val)
                
            elif val.find('infrastructure')!=-1 or val.find('cisco')!=-1  or val.find('vmware')!=-1 :
                job = "Infrastructures"
                # print(str(lng) + "  "+val)
                
            else:
                job = "Net other"  
                # print(str(lng) + "  "+val)
            
        #Ce sont les intitulés en rapport avec le support aux entreprises   
        elif val.find('quality')!=-1 or val.find('risque')!=-1 or val.find('risques')!=-1 or val.find('recette')!=-1 or val.find('grands comptes')!=-1 or val.find('avant-vente')!=-1 or val.find('production')!=-1  or val.find('support')!=-1  or val.find('support')!=-1 or val.find('projets')!=-1 or val.find('architecte')!=-1 or val.find('technique')!=-1 or val.find('investor')!=-1 or val.find('qa')!=-1 or val.find('salesforce')!=-1 or  val.find('supply chain')!=-1  or val.find('crm')!=-1:
            
            class_job="Support"
            # print(str(lng) + "  "+val)
            
            if val.find('projet')!=-1 or val.find('architecte')!=-1:
                job = "Chef de projets" 
                # print(str(lng) + "  "+val)
                
            elif val.find('qa')!=-1 :
                job = "Responsable qualité" 
                # print(str(lng) + "  "+val)
                
            elif val.find('risques')!=-1 or val.find('expert')!=-1:
                job = "Auditeur/Expert" 
                # print(str(lng) + "  "+val)
                
            elif val.find('avant-vente')!=-1 or val.find('salesforce')!=-1  or val.find('grands comptes')!=-1:
                job = "Commercial" 
                # print(str(lng) + "  "+val)
                
            elif val.find('maintenance')!=-1 or val.find('production')!=-1  or val.find('relations')!=-1:
                job = "Operationnel" 
                # print(str(lng) + "  "+val)
                
            else:
                job = "Support other"  
                #print(str(lng) + "  "+val)
            
        #Ce sont les rejets qui n'ont pas pu être labélisées   
        else:
            x+=1
            class_job="Other"
            job = "Other"  
            # print(str(lng) + "  "+val)
            pass
        
        
             
        if val.find('technicien')!=-1 or val.find('contrôleur')!=-1 or val.find('technical')!=-1:
            niveau='Technicien'
            #print(str(lng) + "  "+val)
            
        elif val.find('doctorant')!=-1:
            niveau = "Doctorant" 
            # print(str(lng) + "  "+val)
            
        elif val.find('ingénieur')!=-1 or val.find('ingenieur')!=-1 or val.find('concepteur')!=-1 or val.find('engineer')!=-1:
            niveau = "Ingénieur" 
            # print(str(lng) + "  "+val)
            
        elif val.find('expert')!=-1 or val.find('consultant')!=-1  or val.find('auditeur')!=-1 or val.find('spécialiste')!=-1 :
            niveau = "Expert" 
            # print(str(lng) + "  "+val)
           
        elif val.find('développeur')!=-1 or val.find('developpeur')!=-1 or val.find('developer')!=-1  :
            niveau = "Développeur" 
            # print(str(lng) + "  "+val)
            
        elif val.find('responsable')!=-1 or val.find('chef de projet')!=-1 or val.find('manager')!=-1 or val.find('référent')!=-1:
            niveau = "Managment" 
            # print(str(lng) + "  "+val)
                
        elif val.find('analyst')!=-1 :
            niveau = "Analyst" 
            # print(str(lng) + "  "+val)
            
        elif val.find('scientist')!=-1 :
            niveau = "Scientist" 
            # print(str(lng) + "  "+val)
            
        elif val.find('architecte')!=-1 :
            niveau = "Architecte" 
            # print(str(lng) + "  "+val)
            
        elif val.find('stage')!=-1 or  val.find('alternance')!=-1 or  val.find('apprenti')!=-1:
            niveau = "Stage/Alternance/Apprenti" 
            # print(str(lng) + "  "+val)
            
        else:
            niveau='Autre'
            print(str(lng) + "  "+val)
            pass
        
        
        df1=pd.DataFrame([pd.Series([job, class_job, niveau])])
        df2= pd.concat([df2,df1], ignore_index=True)
    return df2


## **Main**

---



In [ ]:
df_lieu=extract_jsonTab(tab['lieu'], 2)
df_post=extract_jsonTab(tab['Type de poste'],8)
df_competences=extract_jsonTab(tab['competences'],5)
df_intitule_poste=extract_jsonTab(tab['Intitulé du poste'], 1)

#frames = []
#for i in range(len(df_competences.columns)):
#    frames.append(pd.get_dummies(df_competences[i]))
#    print(sum(pd.get_dummies(df_competences[i]).sum()))

df_competences_serialised = serialisation_colonnes_competences(df_competences)

In [ ]:
date_pub = réecriture_des_dates_de_publication(tab['Date de publication'])

df_salaireMin=recuperation_salaire_minimum(df_lieu[1])
df_salaireMax=recuperation_salaire_maximum(df_lieu[1])

df_intitule=labelisations_colonne_intitule_poste(tab)
df_intitule

In [ ]:
#options pour voir l'ensemble du dataframe dans Colab
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
#extraction des colonnes
lieu = df_lieu[0]
société = df_post[2]
recruteur = df_post[4]
contrat1 = df_post[3]
contrat2 = df_post[5]
contrat3 = df_post[7]
contrat = contrat1 + contrat2 + contrat3

In [ ]:
#création du dataframe final
df = pd.concat([date_pub, df_intitule_poste, df_competences_serialised, lieu, df_salaireMin, df_salaireMax, contrat, société], axis=1)
df.columns = ['Date de publication', 'Intitulé du poste', 'Compétences', 'Lieu', 'Salaire minimum', 'Salaire maximum', 'Type de contrat', 'Nom de la société']

In [ ]:
# Formatage de la colonne 'Type de contrat'
df = df.astype(str)
df['Type de contrat'] = df['Type de contrat'].str.replace(' - temps plein','')

In [ ]:
# Formatage de la colonne 'Nom de la société'
df['Nom de la société'] = df['Nom de la société'].str.lower()
df['Nom de la société'] = df['Nom de la société'].str.replace(' sas','')
df['Nom de la société'] = df['Nom de la société'].str.replace(' s.a.s.','')
df['Nom de la société'] = df['Nom de la société'].str.replace(' group','')
df['Nom de la société'] = df['Nom de la société'].str.replace('-group','')
df['Nom de la société'] = df['Nom de la société'].str.replace('groupe ','')
df['Nom de la société'] = df['Nom de la société'].str.replace('.com','')
df['Nom de la société'] = df['Nom de la société'].str.replace(' & ','&')
def remove_accents(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')
df['Nom de la société'] = df['Nom de la société'].apply(remove_accents)

In [ ]:
# Formatage de la colonne 'Lieu'
# Import d'un fichier csv avec les noms de toutes les communes d'Ile-de-France
# https://www.data.gouv.fr/fr/datasets/les-communes-d-ile-de-france-idf/ (conserver seulement les colomnes 'nomcom' et 'numdep')
idf_data = pd.read_csv('communes_ile-de-france.csv')
idf_data.head()

In [ ]:
# Formatage des noms de villes.
idf_data['nomcom'] = idf_data['nomcom'].str.lower()
idf_data['nomcom'] = idf_data['nomcom'].str.replace(' ','-')
idf_data.head()

In [ ]:
# Création d'une liste de villes à partir de la colonne 'nomcom'.
villes = idf_data['nomcom'].tolist()
# La Défense n'est pas vraiment une commune, mais peut être rajoutée manuellement.
villes.append('la-defense')
villes[:10]

In [ ]:
# Formatage des noms de villes dans la colomne 'Lieu' de notre dataframe principal.
df['Lieu'] = df['Lieu'].str.lower()
df['Lieu'] = df['Lieu'].str.replace(' ','-')
df.head()

In [ ]:
# Filtrage de la colonne 'Lieu' pour conserver soit le nom de la ville, soit le département.
departements = ['paris', 'seine-et-marne', 'yvelines', 'essonne', 'hauts-de-seine', 'seine-saint-denis', 'val-de-marne', "val-d'oise"]
def clean_lieu(lieu):
  for ville in villes:
    if ville in lieu:
      return ville
    else:
      for departement in departements:
        if departement in lieu:
          return departement
df['Lieu'] = df['Lieu'].apply(clean_lieu)
df['Lieu'].fillna('île-de-france', inplace=True) 

In [ ]:
# Formatage de la colonne 'Compétences'
def clean_comp(comp):
    return comp.rstrip(', ')
df['Compétences'] = df['Compétences'].apply(clean_comp)
df['Compétences'] = df['Compétences'].str.replace('réseaux','réseau')

## Tableau nettoyé
---

In [ ]:
df

## Création d'un csv / récupération d'un path
---

In [ ]:
df.to_csv('df_travail.csv', index = False)

path = os.getcwd()
print(f"Le répertoire courant est : {path} \n")